In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [2]:
# CHANGE ME!
TASK_ID = 'customer_orders'
AUTHOR = 'g'

In [3]:
task = Task(
    id=TASK_ID,
    description="Select all order ids from customers that live in California",
    name="Inner join on customers and orders",
    category="Aggregation",
    plan=[
        {
        "id": "order_ids",
        "description": "select order ids",
    }, {
        "id": "California",
        "description": "customer is in California",
    }, {
        "id": "join",
        "description": "join the two tables together",
    },
    ],
    sample_input={
       "customers":[{"cid":0,"customer":"A","location":"California"},{"cid":1,"customer":"B","location":"California"},
                    {"cid":2,"customer":"C","location":"Virginia"},{"cid":3,"customer":"D","location":"New York"}],
        "orders":[{"cid":0,"oid":4,"customer":"A","orderNum":"A1"},{"cid":0,"oid":5,"customer":"A","orderNum":"A2"},
                  {"cid":1,"oid":6,"customer":"B","orderNum":"B1"},
                 {"cid":1,"oid":7,"customer":"B","orderNum":"B2"},{"cid":2,"oid":8,"customer":"C","orderNum":"C1"}],
    },
    sample_output=[4,5,6,7],
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [4]:
sql = replace(prototype,
    language='sql',
    source='''  
SELECT orders.oid 
FROM 
(orders INNER JOIN customers ON
orders.cid = customers.cid) 
WHERE location="California"
''')
sql.execute(task)
sql.save()

In [5]:
datalog = replace(prototype,
    language='datalog',
    source='''  
customer_orders(oid):-
  customers(cid,customer,"California"),
  orders(cid,_,oid,_).
''')
datalog.execute(task)
datalog.save()

In [6]:
pandas = replace(prototype,
    language='python-pandas',
    source='''  
def customer_orders(customers,orders):
  joined = orders.join(customers, on="cid",how="inner",lsuffix="_j")
  joined = joined.loc[joined['location'] == "California"]
  return joined.oid.tolist()

''')
pandas.execute(task)
pandas.save()

In [7]:
imp = replace(prototype,
    language='python-imperative',
    source='''  
def customer_orders(customers,orders):
  oids = []
  for order in orders:
    for customer in customers:
      if customer['cid'] == order['cid']:
        if customer['location'] == "California":
          oids.append(order['oid'])
  return oids
''')
imp.execute(task)
imp.save()

In [8]:
func = replace(prototype,
    language='python-functional',
    source='''  
def customer_orders(customers,orders):
  oids = []
  cids = [customer['cid'] for customer in customers
    if customer['location'] == "California"]
  oids = [order['oid'] for order in orders 
    if order['cid'] in cids]
  return oids
''').load_plan()
func.execute(task)
func.save()

In [17]:
program = imperative
program.widget(task)

Output()

CodeWidget(program='{"task": "inner_join", "language": "python-imperative", "plan": {"California": [{"line": 3…